In [1]:
import os
import logging
from glob import glob
from datetime import datetime
from collections import namedtuple
from pickle import dump, load

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import transformers
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer, EarlyStoppingCallback, BatchEncoding
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
import optuna

from arabert.preprocess import ArabertPreprocessor
from utilities import *

# Parameters

In [12]:
# Preprocessing params
sequence_length: int = 32 # Not sure what happens when setting it with a number different from what its trained on

# Model 
model_name: str = "aubmindlab/bert-large-arabertv2"
pretrained_classifier_name: str = "2021-12-09-train-0.963134765625"

# Data
df: pd.DataFrame = get_annotated_data_folder_data()

# Paths
code_folder_path: str = ""

# Training 
batch_size: int = 64

# Etc
open_tensorboard: bool = True

## Don't touch

In [13]:
# Preprocessing
label2id = pretrained_classifier.config.label2id
id2label = pretrained_classifier.config.id2label

# Model
pretrained_classifier_path = join(code_folder_path, "models", "finalized_models", pretrained_classifier_name) 
pretrained_classifier = AutoModelForSequenceClassification.from_pretrained(pretrained_classifier_path)
arabert_prep = ArabertPreprocessor(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2021-12-11 18:07:25,695 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


# Preprocessing & loading model

In [17]:
%%time

df["Labels"] = df["Region"].apply(label2id.get)
df["Text"] = df["Text"].apply(arabert_prep.preprocess)
df_encoding = tokenize(tokenizer, df["Text"].to_list(), sequence_length)

test_set = Dialect_dataset(df_encoding, df["Labels"].to_list())

Wall time: 27.4 s


# Testing

In [19]:
trainer = Trainer(model=pretrained_classifier, compute_metrics=compute_metrics)
prediction = trainer.predict(test_set)
prediction.metrics

***** Running Prediction *****
  Num examples = 26804
  Batch size = 8
100%|█████████▉| 3349/3351 [02:53<00:00, 20.45it/s]D:\Softwarez\Anaconda\envs\graduation_project\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 2.460561752319336,
 'test_macro_f1': 0.060848385158004614,
 'test_macro_precision': 0.0358827040740188,
 'test_macro_recall': 0.2,
 'test_accuracy': 0.179413520370094,
 'test_runtime': 198.7829,
 'test_samples_per_second': 134.841,
 'test_steps_per_second': 16.858}

100%|██████████| 3351/3351 [03:10<00:00, 20.45it/s]

In [20]:
y_pred = [np.argmax(pred) for pred in prediction[0]]
y_true = df["Labels"]
accuracy_score(y_true , y_pred)

0.179413520370094

In [21]:
text = "والله مابعرف اغلب الناس رواتبها ومش ععارفين كيف ممشين هاذ بوخذ الاف وعدا عن السيارات والبدل والسفرات والخ وبحكيلك قليل هو الفساد بيجي من قليل" 
predict_dialect(model_path, text, tokenizer, arabert_prep.preprocess, sequence_length)

NameError: name 'model_path' is not defined